<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment

# 카이제곱검정


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving df.csv to df.csv


In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['df.csv']), index_col=0, encoding = 'cp949')
df = df.replace({'-':0})
df.index
df

,부문(1),60㎡이하,60∼85㎡,85㎡초과
City,,,,
서울,민간부문,54,2,0
부산,민간부문,577,735,142
대구,민간부문,143,1437,44
대전,민간부문,782,1,0


In [ ]:
df = df.rename({'60㎡이하':'~60m2', '85㎡초과':'85m2~'}, axis='columns')
df = df.drop(['부문(1)'], axis=1)
df

,~60m2,60∼85㎡,85m2~
City,,,
서울,54,2,0
부산,577,735,142
대구,143,1437,44
대전,782,1,0


In [ ]:
df.shape

(4, 3)

In [ ]:
df.sum().sum()

3917

## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

예시) 만약 **9월달 데이터**를 기준으로 한다면
```python
 [0+0+0+52+2+0, 0+0+0+590+665+142, 0+0+0+113+1061+42, 0+0+0+772+1+0]
```
을 비교 하게 될 것입니다.




In [ ]:
se = 0+0+0+54+2+0
bs = 0+0+0+577+735+142
dg = 0+0+0+143+1437+44
dj = 0+0+0+782+1+0

observed = se, bs, dg ,dj
observed

expected= sum(observed)/4
ex = expected
ex

979.25

In [ ]:
se, bs, dg, dj

(56, 1454, 1624, 783)

In [ ]:
import numpy as np
from scipy.stats import chisquare
obs = np.array([56, 1454, 1624, 783])
expected = np.array([979.25, 979.25, 979.25, 979.25])
chi1 = chisquare(obs, expected)

In [ ]:
chi1.statistic

1564.4572376818994

In [ ]:
# 설명
# 귀무가설: 지역에 따라 분양은 차이가 없을 것이다
# 대립가설: 지역에 따라 분양은 차이가 있을 것이다
# pvalue가 0.0이므로 지역에 따라 미분양은 차이가 있다

## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

예시) **9월달 데이터**를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




In [ ]:
df

,~60m2,60∼85㎡,85m2~
City,,,
서울,54,2,0
부산,577,735,142
대구,143,1437,44
대전,782,1,0


In [ ]:
from scipy.stats import chi2_contingency
obs = np.array([[54,2,0], [577,735,142], [143,1437,44], [782,1,0]])
exp = np.array([979.25, 979.25, 979.25, 979.25])
total_obs = np.sum(obs)
total_exp = np.sum(exp)

In [ ]:
chi2 = chi2_contingency(obs)
chi2


(2064.576731417199, 0.0, 6, array([[ 22.24559612,  31.09522594,   2.65917794],
        [577.59101353, 807.36533061,  69.04365586],
        [645.12228746, 901.76155221,  77.11616033],
        [311.04110288, 434.77789124,  37.18100587]]))

In [ ]:
#pvalue가 0.0이므로 지역에 따라 차이가 있다 

### 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.

이 후 `chi2_contingency()`의 결과물과 비교해보세요

In [ ]:
df

,~60m2,60∼85㎡,85m2~
City,,,
서울,54,2,0
부산,577,735,142
대구,143,1437,44
대전,782,1,0


In [ ]:
obs = np.array([56, 1454, 1624, 783])
exp = np.sum(obs)/len(obs)
chi = np.sum(np.power(obs - exp, 2)/exp)
chi

1564.4572376818994

In [ ]:
pval = 1 - stats.chi2.cdf(chi, df = len(obs)-1) #len(obs) - data 길이
pval

0.0

In [ ]:
chi == chi1[0], pval == chi1[1]

(True, True)

In [ ]:
#scipy & numpy 둘 다 같은 값 나옴

## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [ ]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]

def myChisq(value):
    exp = np.sum(value)/len(value)
    chi = np.sum(np.power(value-exp, 2)/exp)
    pval = 1 - stats.chi2.cdf(chi, df = len(value)-1)
    print(value,"의 값 출력")
    print('chisquare = ' ,chi)
    print('pvalue = ', pval)
    return None

myChisq(v1) 

# chisquare값과 pvalue를 출력해야합니다.

myChisq(v2)

[18, 22, 20, 15, 23, 22] 의 값 출력
chisquare =  2.3000000000000003
pvalue =  0.8062668698851285
[5, 23, 26, 19, 24, 23] 의 값 출력
chisquare =  14.8
pvalue =  0.011251979028327308


### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



In [ ]:
import scipy.stats as stats
import pandas as pd
import urllib
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

a = [38,33,35,92,76,97,88,41,11,9]
b = [18,52,62,48,30,40,87,12,97,82]
c = [28,90,5,49,66,73,96,80,4,17]
d = [8,99,4,12,7,64,18,10,9,20]

df1 = pd.DataFrame({'A':a, 'B':b, 'C':c, 'D':d})

F_statistic, pVal = stats.f_oneway(a,b,c,d)
print('데이터의 일원분산분석 결과 : F={0:.1f}, p={1:.5f}'.format(F_statistic, pVal))

데이터의 일원분산분석 결과 : F=1.7, p=0.17921


In [ ]:
p = 0.17921
if p < 0.05:
  print('평균의 차이가 있다')
else:
  print('평균의 차이가 없을수도 있다')
#pvalue가 5보다 크므로 평균에 차이가 없다는 가설은 기각 할 수 없다.

평균의 차이가 없을수도 있다


## 3. Sign Test

아래 링크를 참조하여 Sign Test에 대한 글을 읽고

<https://en.wikipedia.org/wiki/Sign_test>

다음 데이터에서 두 그룹간에 차이가 있다에 대한 Sign Test의 과정과 결과를 설명하세요

|A|B|
|:-:|:-:|
|142|138|
|140|136|
|144|147|	
|144|139|	
|142|143|	
|146|141|	
|149|143|	
|150|145|	
|142|136|	
|148|146|	



In [ ]:
from scipy import stats
import matplotlib.pyplot as plt

dat_num = {'A':[142,140,144,144,142,146,149,150,142,148],
       'B':[138,136,147,139,143,141,143,145,136,146]}

dat = pd.DataFrame(dat_num)

dat


dat[['A','B']].describe()

dat['result'] = dat['A'] - dat['B'] 
dat['sign'] = 0
dat.loc[dat['result'] > 0,'sign'] = '+'
dat.loc[dat['result'] == 0,'sign'] = '='
dat.loc[dat['result'] < 0,'sign'] = '-'


In [ ]:
dat

,A,B,result,sign
0,142,138,4,+
1,140,136,4,+
2,144,147,-3,-
3,144,139,5,+
4,142,143,-1,-
5,146,141,5,+
6,149,143,6,+
7,150,145,5,+
8,142,136,6,+
9,148,146,2,+


In [ ]:
print(len(dat[dat['sign'] == '+']))

8


In [ ]:
print(len(dat[dat['sign'] == '-']))

2


In [ ]:
dat_pvalue = (0.001+0.010+0.044)
dat_pvalue
#Pvalue가 0.5보다 크므로, 귀무가설은 기각 X
#A B 간에 차이가 있다고 볼 수 없다

0.05499999999999999